In [10]:
import os
import cv2
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import gradio as gr
from tensorflow.keras.models import load_model
from skimage.measure import label

In [11]:
# Paths
input_dir = "/content/train/images/"  # Directory with original images
output_dir = "/content/results"  # Directory where segmented masks will be saved

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to segment an image (you can modify this function to use other segmentation methods)
def segment_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply threshold to create a binary mask
    _, binary_mask = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    return binary_mask

# Loop over all images in the input directory
for image_name in os.listdir(input_dir):
    image_path = os.path.join(input_dir, image_name)

    # Read the image
    image = io.imread(image_path)

    # Segment the image
    segmented_mask = segment_image(image)

    # Save the segmented mask
    output_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_path, segmented_mask)
print(f"Segmented masks saved in {output_dir}")


Segmented masks saved in /content/results


In [16]:
# Paths
train_images_dir = '/content/train/images/'
train_masks_dir = '/content/results/'

# Function to load images and masks, and resize them to the desired shape
def load_images_and_masks(image_dir, mask_dir, target_size=(256, 256)):
    images = []
    masks = []

    for image_name in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_name)
        mask_path = os.path.join(mask_dir, image_name)

        # Load image and mask
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Resize the image and mask to the target size
        image = cv2.resize(image, target_size)
        mask = cv2.resize(mask, target_size)

        # Normalize images and masks
        image = image / 255.0
        mask = mask / 255.0

        images.append(image)
        masks.append(mask)

    return np.array(images), np.array(masks)

# Load training data with resized images
X_train, y_train = load_images_and_masks(train_images_dir, train_masks_dir, target_size=(256, 256))

# Expand dimensions of masks to match the expected shape of the model (batch, height, width, channels)
y_train = np.expand_dims(y_train, axis=-1)

# U-Net architecture
def unet_model(input_shape):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    b = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    b = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(b)

    # Decoder
    u1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(b)
    u1 = layers.concatenate([u1, c2])
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)

    u2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c3)
    u2 = layers.concatenate([u2, c1])
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c4)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

# Create the model
input_shape = (256, 256, 3)  # Adjust based on your image dimensions
model = unet_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=16, epochs=1, validation_split=0.1)

# Save the trained model
model.save('/content/unet_model.h5')

print("Model training complete.")


315/315 [==============================] - 1539s 5s/step - loss: 0.1234 - accuracy: 0.9350 - val_loss: 0.0536 - val_accuracy: 0.9587
Model training complete.


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
# Load the pre-trained U-Net model
model = load_model('/content/unet_model.h5')

# Function to preprocess the input image
def preprocess_image(image, target_size=(256, 256)):
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize
    return np.expand_dims(image, axis=0)  # Add batch dimension

# Function to post-process and count buildings
def postprocess_and_count(mask):
    # Threshold the mask to create a binary image
    binary_mask = mask > 0.5

    # Label connected regions (buildings)
    labeled_mask, num_buildings = label(binary_mask, return_num=True)

    return labeled_mask, num_buildings

# Function to detect buildings in an image and return the count
def detect_buildings(image):
    # Preprocess the image for the model
    input_image = preprocess_image(image)

    # Predict the mask using the model
    predicted_mask = model.predict(input_image)[0, :, :, 0]

    # Post-process the mask and count the buildings
    labeled_mask, num_buildings = postprocess_and_count(predicted_mask)

    # Convert labeled mask back to RGB for visualization
    labeled_mask_rgb = cv2.cvtColor((labeled_mask > 0).astype(np.uint8) * 255, cv2.COLOR_GRAY2RGB)

    return labeled_mask_rgb, f"Number of buildings detected: {num_buildings}"

# Gradio interface
gr_interface = gr.Interface(
    fn=detect_buildings,
    inputs=gr.Image(type="numpy"),
    outputs=[gr.Image(type="numpy", label="Building Segmentation"), gr.Textbox(label="Building Count")],
    title="Building Detection and Counting",
    description="Upload a satellite image to detect and count buildings.",
)

# Launch the Gradio interface
gr_interface.launch(share=True, show_error=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://512acf1f52a07a4e25.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 89ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://512acf1f52a07a4e25.gradio.live


In [ ]:
import numpy as np
import cv2
import gradio as gr
from tensorflow.keras.models import load_model
from skimage.measure import label, regionprops

# Load the pre-trained U-Net model
model = load_model('/content/unet_model.h5')

# Function to preprocess the input image
def preprocess_image(image, target_size=(256, 256)):
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize
    return np.expand_dims(image, axis=0)  # Add batch dimension

# Function to estimate population based on building count
def estimate_population(building_count, people_per_building=5):
    return building_count * people_per_building

# Function to post-process and count buildings
def postprocess_and_count(mask):
    # Threshold the mask to create a binary image
    binary_mask = mask > 0.5

    # Label connected regions (buildings)
    labeled_mask, num_buildings = label(binary_mask, return_num=True)

    return labeled_mask, num_buildings

# Function to draw boundaries around detected buildings
def draw_building_boundaries(image, labeled_mask):
    contours, _ = cv2.findContours((labeled_mask > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    outlined_image = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 2)
    return outlined_image

# Function to detect buildings, estimate population, and draw boundaries
def detect_buildings(image):
    # Preprocess the image for the model
    input_image = preprocess_image(image)

    # Predict the mask using the model
    predicted_mask = model.predict(input_image)[0, :, :, 0]

    # Post-process the mask and count the buildings
    labeled_mask, num_buildings = postprocess_and_count(predicted_mask)

    # Estimate the population based on building count
    estimated_population = estimate_population(num_buildings)

    # Draw boundaries around detected buildings
    outlined_image = draw_building_boundaries(cv2.resize(image, (256, 256)), labeled_mask)

    # Convert labeled mask back to RGB for visualization
    labeled_mask_rgb = cv2.cvtColor((labeled_mask > 0).astype(np.uint8) * 255, cv2.COLOR_GRAY2RGB)

    return outlined_image, f"Number of buildings detected: {num_buildings}", f"Estimated population: {estimated_population}"

# Gradio interface
gr_interface = gr.Interface(
    fn=detect_buildings,
    inputs=gr.Image(type="numpy"),
    outputs=[
        gr.Image(type="numpy", label="Building Segmentation with Boundaries"),
        gr.Textbox(label="Building Count"),
        gr.Textbox(label="Estimated Population")
    ],
    title="Building Detection, Counting, and Population Estimation",
    description="Upload a satellite image to detect, count buildings, and estimate population based on building count.",
)

# Launch the Gradio interface
gr_interface.launch(share=True, show_error=True, debug=True)
